In [ ]:
import sys
 
sys.path.append('..')
from piece import piece
piece.start()

In [ ]:
piece.reset()
piece.synth_player.start()
piece.synth_player.note() \
    .sine(piece.control_instruments.static_control(440), piece.control_instruments.sine_control(0, 1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(0, 1)

piece.synth_player.note() \
    .sine(piece.control_instruments.static_control(220), piece.control_instruments.sine_control(0, 1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(1, 1)
    


In [ ]:
piece.reset()
piece.synth_player \
    .add_sound("wood_hit_1", "/Users/danielstahl/Documents/Projects/py-soundmining-tools/samples/Wood hit 1.flac", 0.201, 0.339) \
    .start()

In [ ]:
piece.synth_player.note() \
    .sound_mono("wood_hit_1", 1, piece.control_instruments.static_control(1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(0)

In [ ]:
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        print(f"Patch arguments {patch_arguments}")

my_handler = MyHandler(piece.supercollider_client)

piece.receiver.set_note_handler(my_handler)

In [ ]:
piece.stop()

In [ ]:
from math import pi

from ipycanvas import Canvas

canvas = Canvas(width=200, height=200)

canvas.fill_style = "red"
canvas.stroke_style = "blue"

canvas.fill_arc(60, 60, 50, 0, pi)
canvas.stroke_circle(60, 60, 40)

canvas

In [ ]:
import matplotlib.pyplot as plt

from soundmining_tools import spectrum
from soundmining_tools import note
d2 = note.note_to_hertz("d2")
h2 = note.note_to_hertz("h2")
fact = spectrum.make_fact(d2, h2)
spect = spectrum.make_spectrum(d2, fact, 15)
fig, ax = plt.subplots()
x_values = list(range(1, 16))
ax.scatter(x_values, spect)
plt.show()


In [ ]:
from soundmining_tools import spectrum
from soundmining_tools import note
from soundmining_tools.sieve import *

d2 = note.note_to_hertz("d2")
h2 = note.note_to_hertz("h2")
fact = spectrum.make_fact(d2, h2)
spect = spectrum.make_spectrum(d2, fact, 15)

sieve = UnionSieve([SimpleSieve(3, 0), SimpleSieve(4, 1)])
amp_values = [1.0 / (i + 1.0) if sieve.is_sieve(i) else 0.0 for i in range(15)]
phases = [0.0 for i in range(15)]

piece.reset()
piece.synth_player.start()
piece.synth_player.note() \
    .bank_of_osc(spect, amp_values, phases) \
    .mono_volume(piece.control_instruments.sine_control(0, 1)) \
    .pan(piece.control_instruments.static_control(0)) \
    .play(0, 2)

    

In [ ]:
# ipwidgets https://ipywidgets.readthedocs.io/en/stable/


In [ ]:
from soundmining_tools import note, spectrum
from soundmining_tools.sieve import SimpleSieve, UnionSieve
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments

fundamental = note.note_to_hertz("a1")
first_partial = note.note_to_hertz("h2")
scale_size = 4
fact = spectrum.make_fact(fundamental, first_partial)
fundamental_cents = note.hertz_to_cents(fundamental)
octave_cents = note.hertz_to_cents(first_partial) - fundamental_cents
note_cents = octave_cents / scale_size
scale = [note.cents_to_hertz(fundamental_cents + (note_cents * i)) for i in range(8)]
scale_spectrums = [spectrum.make_spectrum(note, fact, 15) for note in scale]

sieve = UnionSieve([SimpleSieve(3, 0), SimpleSieve(4, 1)])

amp_values = [1.0 / (i + 1.0) if sieve.is_sieve(i) else 0.0 for i in range(15)]
phases = [0.0 for i in range(15)]

piece.reset()
piece.synth_player.start()

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        spect = scale_spectrums[patch_arguments.note]
        piece.synth_player.note() \
            .bank_of_osc(spect, amp_values, phases) \
            .high_pass_filter(piece.control_instruments.static_control(spect[2])) \
            .low_pass_filter(piece.control_instruments.static_control(spect[7])) \
            .mono_volume(piece.control_instruments.three_block_control((0, 0.7, 0.5, 0), (0.1, 0.5, 0.4), (0, 0, 0))) \
            .pan(piece.control_instruments.static_control(0)) \
            .play(0, 2)

my_handler = MyHandler(piece.supercollider_client)

piece.receiver.set_note_handler(my_handler)
